# IP Routing

The network topology consists of one router called **r1** and two hosts, **h1** and **h2**. Their IP and MAC addresses can be found below.

![title](imgs/routing.png)

The code and steps presented below will establish communication between **h1** and **h2**.

In [ ]:
#!/usr/bin/python

import os

from mininet.log import setLogLevel, info
from mn_wifi.cli import CLI
from mn_wifi.net import Mininet_wifi
from mn_wifi.bmv2 import P4Switch


def topology():
    'Create a network.'
    net = Mininet_wifi()

    info('*** Adding stations/hosts\n')
    h1 = net.addHost('h1', ip='10.0.1.1/24', mac="00:00:00:00:00:01")
    h2 = net.addHost('h2', ip='10.0.2.1/24', mac="00:00:00:00:00:02")

    path = os.path.dirname(os.getcwd())
    json_file = path + '/p4-scenarios/routing.json'

    info('*** Adding P4 Switch\n')
    r1 = net.addSwitch('r1', cls=P4Switch, netcfg=True,
                       json=json_file, thriftport=50001)

    info('*** Creating links\n')
    net.addLink(h1, r1)
    net.addLink(h2, r1)

    info('*** Starting network\n')
    net.start()

    info('*** Running CLI\n')
    CLI(net)

    info('*** Stopping network\n')
    net.stop()


if __name__ == '__main__':
    setLogLevel('info')
    topology()

*** Adding stations/hosts
*** Adding P4 Switch
*** Creating links
*** Starting network
*** Starting controller(s)

*** Starting L2 nodes
r1 

....⚡️ simple_switch_grpc @ 43699 thrift @ 50001



*** Running CLI
*** Starting CLI:


mininet-wifi> r1 simple_switch_CLI --thrift-port 50001 <<< "table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.1/32 => 00:00:00:00:00:01 1"


Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-0a:00:01:01/32
action:              ipv4_forward
runtime data:        00:00:00:00:00:01	00:01
Entry has been added with handle 0
RuntimeCmd: 


...

4e:64:f9:9f:dd:3f


mininet-wifi> py r1.intfs[2].mac


7a:c7:f0:2f:c1:1b


mininet-wifi> h1 arp -i h1-eth0 -s 10.0.1.254 4e:64:f9:9f:dd:3f
mininet-wifi> h2 arp -i h2-eth0 -s 10.0.2.254 7a:c7:f0:2f:c1:1b
mininet-wifi> h1 ping -c1 h2


PING 10.0.2.1 (10.0.2.1) 56(84) bytes of data.
64 bytes from 10.0.2.1: icmp_seq=1 ttl=63 time=4.30 ms

--- 10.0.2.1 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 4.304/4.304/4.304/0.000 ms


**Great work!**